In [ ]:
#!pip install python-dotenv
#!pip install openai

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

## Chat API : OpenAI

Let's start with a direct API calls to OpenAI.

In [3]:
def get_completion(prompt, model=llm_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [4]:
get_completion("What is capital of Maharahtra and Gujrat?")

'The capital of Maharashtra is Mumbai and the capital of Gujarat is Gandhinagar.'

In [5]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
style = """American English \
in a calm and respectful tone
"""

In [7]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [8]:
response = get_completion(prompt)

In [9]:
response

"I understand that you are upset about your blender lid flying off and causing a mess in your kitchen. It can be frustrating when things don't go as planned. Unfortunately, it seems that the warranty does not cover the cost of cleaning up the mess. I would be happy to assist you in finding a solution to this problem."

## Chat API : LangChain

Let's try how we can do the same using LangChain.

In [ ]:
#!pip install --upgrade langchain

### Model

In [10]:
from langchain.chat_models import ChatOpenAI

In [11]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-0301', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

### Prompt template

#### Generate MCQ question for student

In [25]:
template_string = """Generate {no_of_que} {level} level MCQ questions \
on the topic {topic}"""

In [26]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [17]:
prompt_template

ChatPromptTemplate(input_variables=['level', 'topic', 'no_of_que'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['level', 'no_of_que', 'topic'], output_parser=None, partial_variables={}, template='Generate {no_of_que} {level} level MCQ questions frin the topic {topic}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [14]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['level', 'no_of_que', 'topic'], output_parser=None, partial_variables={}, template='Generate {no_of_que} {level} level MCQ questions frin the topic {topic}', template_format='f-string', validate_template=True)

In [27]:
prompt_template.messages[0].prompt.input_variables

['level', 'no_of_que', 'topic']

In [16]:
no_of_que = """3"""
level = """advanced"""
topic = """Ionic Equilibrium"""

In [29]:
student_message = prompt_template.format_messages(no_of_que = no_of_que,
                                                 level = level, 
                                                 topic = topic)

In [23]:
print(type(student_message))
print(type(student_message[0]))

<class 'list'>
<class 'langchain.schema.HumanMessage'>


In [30]:
print(student_message[0])

content='Generate 3 advanced level MCQ questions on the topic Ionic Equilibrium' additional_kwargs={} example=False


In [32]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(student_message)

In [33]:
print(customer_response.content)

1. Which of the following statements is true regarding the dissociation of a weak acid in water?
a) The degree of dissociation is directly proportional to the concentration of the acid.
b) The degree of dissociation is inversely proportional to the concentration of the acid.
c) The degree of dissociation is independent of the concentration of the acid.
d) The degree of dissociation is directly proportional to the concentration of the conjugate base.

2. Which of the following factors affect the solubility of a sparingly soluble salt in water?
a) pH of the solution
b) Temperature of the solution
c) Presence of common ions in the solution
d) All of the above

3. Which of the following statements is true regarding the common ion effect?
a) The solubility of a salt decreases in the presence of a common ion.
b) The solubility of a salt increases in the presence of a common ion.
c) The solubility of a salt remains unchanged in the presence of a common ion.
d) The common ion effect is only ob

#### Email-Chatbot for Customer Service

In [42]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""
prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [43]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [44]:
service_style_pirate = """\
a polite tone \
that speaks in Hindi\
"""

In [45]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in Hindi. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! See ya!
```



In [46]:
service_response = chat(service_messages)
print(service_response.content)

अरे वहाँ ग्राहक, आपकी रसोई के सफाई खर्चों को वारंटी नहीं कवर करती क्योंकि आपने अपने ब्लेंडर का उपयोग गलत कर दिया था जब आपने ब्लेंडर चलाने से पहले ढक्कन नहीं लगाया था। बुरा हुआ! फिर मिलते हैं!


## Output Parsers

Let's start with defining how we would like the LLM output to look like:

In [47]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [48]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [49]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})]


In [50]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

{
    "gift": true,
    "delivery_days": 2,
    "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}


In [51]:
type(response.content)

str

In [52]:
# You will get an error by running this line of code 
# because'gift' is not a dictionary
# 'gift' is a string
response.content.get('gift')

AttributeError: 'str' object has no attribute 'get'

In [53]:
type(response.content)

str

### Parse the LLM output string into a Python dictionary

In [54]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [55]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [69]:
type(response_schemas[0])

langchain.output_parsers.structured.ResponseSchema

In [56]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [57]:
format_instructions = output_parser.get_format_instructions()

In [58]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [59]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [60]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "

In [61]:
response = chat(messages)

In [62]:
print(response.content)

```json
{
	"gift": true,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [63]:
output_dict = output_parser.parse(response.content)

In [64]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [65]:
type(output_dict)

dict

In [66]:
output_dict.get('delivery_days')

2

Reminder: Download your notebook to you local computer to save your work.

#### Better Parsing Example

In [70]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [77]:
match_summary = """India defeated Pakistan by 69 runs once again after \
4 years, in Ahmedabad Stadium. Kuldeep Yadav is Player of the Match. \
Virat Kohli completed his 55th ODI Century here in Ahmedabad. \
Congratulations India for winning this game.
"""

In [71]:
winner_schema = ResponseSchema(name="winner",
                             description="Who won the game \
                             among two teams ? If cannot be answered \
                             from given data, output : None")
venue_schema = ResponseSchema(name="venue",
                             description="In which city the game \
                             conducted ? If cannot be answered \
                             from given data, output : None")
margin_schema = ResponseSchema(name="margin",
                             description="What was the margin between \
                             winner and loser of the game ? If cannot \
                             be answered from given data, output : None")

In [72]:
response_schemas = [winner_schema, venue_schema, margin_schema]

In [73]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [74]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='winner', description='Who won the game                              among two teams ? If cannot be answered                              from given data, output : None'), ResponseSchema(name='venue', description='In which city the game                              conducted ? If cannot be answered                              from given data, output : None'), ResponseSchema(name='margin', description='What was the margin between                              winner and loser of the game ? If cannot                              be answered from given data, output : None')])

In [75]:
format_instructions = output_parser.get_format_instructions()

In [76]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"winner": string  // Who won the game                              among two teams ? If cannot be answered                              from given data, output : None\n\t"venue": string  // In which city the game                              conducted ? If cannot be answered                              from given data, output : None\n\t"margin": string  // What was the margin between                              winner and loser of the game ? If cannot                              be answered from given data, output : None\n}\n```'

In [78]:
review_template = """\
For the following text, extract the following information:

winner: Which team won the match? \
If answer can't be found output None.

venue: In which city the game was conducted ?\
If answer can't be found output None.

margin: What was the margin between \
winner and loser of the game ? If cannot \
be answered from given data, output : None

text: {text}

{format_instructions}
"""

In [79]:
prompt = ChatPromptTemplate.from_template(template=review_template)

messages = prompt.format_messages(text=match_summary, 
                                format_instructions=format_instructions)

In [80]:
response = chat(messages)

In [86]:
print(response.content)

```json
{
	"winner": "India",
	"venue": "Ahmedabad",
	"margin": "69 runs"
}
```


In [87]:
output_dict = output_parser.parse(response.content)

In [90]:
print("Winners : ", output_dict.get('winner'))
print("Venue : ", output_dict.get('venue'))
print("Won by : ", output_dict.get('margin'))

Winners :  India
Venue :  Ahmedabad
Won by :  69 runs
